In [2]:
# Mengimpor library mysql.connector dan spaCy
import mysql.connector
import spacy

# Membuat koneksi dengan database MySQL
conn = mysql.connector.connect(
    host="localhost",
    user="root", 
    database="pr_scraping_job_vacancy_indonesia_2023_07", 
    password="", 
)

# Membuat cursor untuk menjalankan query SQL
cursor = conn.cursor()

# Membuat tabel baru untuk menyimpan data lowongan pekerjaan yang telah dianotasi
cursor.execute("""
CREATE TABLE job_annotated (
    id INT PRIMARY KEY,
    text TEXT,
    label TEXT
)
""")

# Membuat fungsi untuk mengambil data lowongan pekerjaan dari tabel asal dan mengembalikan list of tuples yang berisi id, text, dan label
def get_job_data():
    # Mengambil data dari tabel job_original
    cursor.execute("SELECT id, jobDescription FROM jobdescription")
    rows = cursor.fetchall()

    # Memuat model spaCy untuk bahasa Indonesia
    nlp = spacy.load("id_core_news_sm")

    # Membuat list kosong untuk menyimpan hasil
    result = []

    # Melakukan iterasi pada setiap baris data
    for row in rows:
        # Mengambil id dan teks lowongan pekerjaan
        id = row[0]
        text = row[1]

        # Melakukan NER pada teks dengan menggunakan model spaCy
        doc = nlp(text)

        # Membuat list kosong untuk menyimpan label entitas
        label = []

        # Melakukan iterasi pada setiap token dalam dokumen
        for token in doc:
            # Mengambil teks dan label entitas dari token
            token_text = token.text
            token_label = token.ent_type_

            # Menyesuaikan format label entitas sesuai dengan standar BIO
            if token_label == "":
                # Jika token tidak memiliki label entitas, beri label O
                label.append("O")
            elif token == token.sent[0] or doc[token.i - 1].ent_type_ != token_label:
                # Jika token adalah token pertama dalam kalimat atau memiliki label entitas yang berbeda dengan token sebelumnya, beri awalan B-
                label.append("B-" + token_label)
            else:
                # Jika token memiliki label entitas yang sama dengan token sebelumnya, beri awalan I-
                label.append("I-" + token_label)

        # Menggabungkan list of tokens menjadi string dengan spasi sebagai pemisah
        text = " ".join([token.text for token in doc])

        # Menggabungkan list of labels menjadi string dengan spasi sebagai pemisah
        label = " ".join(label)

        # Menambahkan tuple yang berisi id, text, dan label ke list hasil
        result.append((id, text, label))

    # Mengembalikan list hasil sebagai output
    return result

# Membuat fungsi untuk menyimpan data lowongan pekerjaan yang telah dianotasi ke tabel baru
def save_job_data(data):
    # Membuat query SQL untuk memasukkan data ke tabel job_annotated
    query = "INSERT INTO job_annotated (id, text, label) VALUES (%s, %s, %s)"

    # Melakukan iterasi pada setiap tuple data
    for row in data:
        # Menjalankan query SQL dengan menggunakan data dari tuple
        cursor.execute(query, row)

    # Melakukan commit pada koneksi untuk menyimpan perubahan
    conn.commit()

# Menjalankan fungsi get_job_data dan menyimpan hasilnya ke variabel job_data
job_data = get_job_data()

# Menjalankan fungsi save_job_data dengan menggunakan variabel job_data sebagai input
save_job_data(job_data)

# Menutup koneksi dengan database MySQL
conn.close()


OSError: [E050] Can't find model 'id_core_news_sm'. It doesn't seem to be a Python package or a valid path to a data directory.